In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import math

In [3]:
dataFrame = pd.read_csv('XGBoost predictionBags.csv')
dataFrame

,Channel,Region,Display,Colour,Material,Sum of NSQ
0,SIS,Chattisgarh,NF,BLACK,51/49 rPES/PU,0
1,SIS,Chattisgarh,NF,BLACK,60/40 REPREVE/40P POLYE,0
2,SIS,Gujarat,NF,BLACK,51/49 rPES/PU,7
3,SIS,Punjab,NaN,BLACK,51/49 rPES/PU,1
4,SIS,Punjab,NaN,BLACK,100% POLYESTER,2
...,...,...,...,...,...,...
742,FSS,Delhi,Wall,NAVY,100% rPES,1
743,FSS,Delhi,Wall,NAVY,80/20 NYLON/PU,0
744,FSS,Delhi,Wall,PURPLE,100% rPES,0
745,FSS,Delhi,Wall,RAZZLEBERRY,100% rPES,0


#### Check for prev predictionCol

In [4]:
for col in dataFrame.columns:
    if(col=='prediction'):
        dataFrame.drop('prediction',axis = 1,inplace = True)

In [5]:
dataFrame

,Channel,Region,Display,Colour,Material,Sum of NSQ
0,SIS,Chattisgarh,NF,BLACK,51/49 rPES/PU,0
1,SIS,Chattisgarh,NF,BLACK,60/40 REPREVE/40P POLYE,0
2,SIS,Gujarat,NF,BLACK,51/49 rPES/PU,7
3,SIS,Punjab,NaN,BLACK,51/49 rPES/PU,1
4,SIS,Punjab,NaN,BLACK,100% POLYESTER,2
...,...,...,...,...,...,...
742,FSS,Delhi,Wall,NAVY,100% rPES,1
743,FSS,Delhi,Wall,NAVY,80/20 NYLON/PU,0
744,FSS,Delhi,Wall,PURPLE,100% rPES,0
745,FSS,Delhi,Wall,RAZZLEBERRY,100% rPES,0


#### Store it in another frame

In [6]:
temp = dataFrame.copy()
temp

,Channel,Region,Display,Colour,Material,Sum of NSQ
0,SIS,Chattisgarh,NF,BLACK,51/49 rPES/PU,0
1,SIS,Chattisgarh,NF,BLACK,60/40 REPREVE/40P POLYE,0
2,SIS,Gujarat,NF,BLACK,51/49 rPES/PU,7
3,SIS,Punjab,NaN,BLACK,51/49 rPES/PU,1
4,SIS,Punjab,NaN,BLACK,100% POLYESTER,2
...,...,...,...,...,...,...
742,FSS,Delhi,Wall,NAVY,100% rPES,1
743,FSS,Delhi,Wall,NAVY,80/20 NYLON/PU,0
744,FSS,Delhi,Wall,PURPLE,100% rPES,0
745,FSS,Delhi,Wall,RAZZLEBERRY,100% rPES,0


In [7]:
le = LabelEncoder()
dataFrame['Channel'] = le.fit_transform(dataFrame['Channel'])
dataFrame['Region']  = le.fit_transform(dataFrame['Region'])
dataFrame['Display'] = le.fit_transform(dataFrame['Display'])
dataFrame['Colour']  = le.fit_transform(dataFrame['Colour'])
dataFrame['Material']= le.fit_transform(dataFrame['Material'])

In [8]:
dataFrame

,Channel,Region,Display,Colour,Material,Sum of NSQ
0,2,2,1,0,5,0
1,2,2,1,0,7,0
2,2,4,1,0,5,7
3,2,9,5,0,5,1
4,2,9,5,0,1,2
...,...,...,...,...,...,...
742,0,3,3,4,3,1
743,0,3,3,4,9,0
744,0,3,3,5,3,0
745,0,3,3,7,3,0


#### Split test and train data

In [9]:
def train_test_split(data):
    data = data.copy()
    n = len(dataFrame)
    return data[:n],data[:n]

In [10]:
train,test = train_test_split(dataFrame)
train,test

(     Channel  Region  Display  Colour  Material  Sum of NSQ
 0          2       2        1       0         5           0
 1          2       2        1       0         7           0
 2          2       4        1       0         5           7
 3          2       9        5       0         5           1
 4          2       9        5       0         1           2
 ..       ...     ...      ...     ...       ...         ...
 742        0       3        3       4         3           1
 743        0       3        3       4         9           0
 744        0       3        3       5         3           0
 745        0       3        3       7         3           0
 746        0       3        3       8         2           1
 
 [747 rows x 6 columns],
      Channel  Region  Display  Colour  Material  Sum of NSQ
 0          2       2        1       0         5           0
 1          2       2        1       0         7           0
 2          2       4        1       0         5          

#### Determine features and target

In [11]:
FEATURES = ['Channel', 'Region', 'Display', 'Colour', 'Material']
TARGET = ['Sum of NSQ']

In [12]:
X_train = train[FEATURES]
Y_train = train[TARGET]
X_test = test[FEATURES]
Y_test = test[TARGET]

#### Train model

In [13]:
reg = xgb.XGBRegressor(n_estimators = 10000, early_stopping_rounds = 50,learning_rate = 0.5)
reg.fit(X_train,Y_train,eval_set = [(X_train,Y_train),(X_test,Y_test)],verbose = True) 

[0]	validation_0-rmse:5.17733	validation_1-rmse:5.17733
[1]	validation_0-rmse:4.38372	validation_1-rmse:4.38372
[2]	validation_0-rmse:4.01841	validation_1-rmse:4.01841
[3]	validation_0-rmse:3.85796	validation_1-rmse:3.85796
[4]	validation_0-rmse:3.76249	validation_1-rmse:3.76249
[5]	validation_0-rmse:3.71643	validation_1-rmse:3.71643
[6]	validation_0-rmse:3.69199	validation_1-rmse:3.69199
[7]	validation_0-rmse:3.65884	validation_1-rmse:3.65884
[8]	validation_0-rmse:3.61910	validation_1-rmse:3.61910
[9]	validation_0-rmse:3.58284	validation_1-rmse:3.58284
[10]	validation_0-rmse:3.56482	validation_1-rmse:3.56482
[11]	validation_0-rmse:3.54966	validation_1-rmse:3.54966
[12]	validation_0-rmse:3.52481	validation_1-rmse:3.52481
[13]	validation_0-rmse:3.51970	validation_1-rmse:3.51970
[14]	validation_0-rmse:3.50997	validation_1-rmse:3.50997
[15]	validation_0-rmse:3.49392	validation_1-rmse:3.49392
[16]	validation_0-rmse:3.49128	validation_1-rmse:3.49128
[17]	validation_0-rmse:3.48194	validation

[144]	validation_0-rmse:3.40373	validation_1-rmse:3.40373
[145]	validation_0-rmse:3.40373	validation_1-rmse:3.40373
[146]	validation_0-rmse:3.40372	validation_1-rmse:3.40372
[147]	validation_0-rmse:3.40372	validation_1-rmse:3.40372
[148]	validation_0-rmse:3.40372	validation_1-rmse:3.40372
[149]	validation_0-rmse:3.40371	validation_1-rmse:3.40371
[150]	validation_0-rmse:3.40369	validation_1-rmse:3.40369
[151]	validation_0-rmse:3.40364	validation_1-rmse:3.40364
[152]	validation_0-rmse:3.40363	validation_1-rmse:3.40363
[153]	validation_0-rmse:3.40362	validation_1-rmse:3.40362
[154]	validation_0-rmse:3.40361	validation_1-rmse:3.40361
[155]	validation_0-rmse:3.40360	validation_1-rmse:3.40360
[156]	validation_0-rmse:3.40360	validation_1-rmse:3.40360
[157]	validation_0-rmse:3.40359	validation_1-rmse:3.40359
[158]	validation_0-rmse:3.40359	validation_1-rmse:3.40359
[159]	validation_0-rmse:3.40359	validation_1-rmse:3.40359
[160]	validation_0-rmse:3.40358	validation_1-rmse:3.40358
[161]	validati

[286]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[287]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[288]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[289]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[290]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[291]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[292]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[293]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[294]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[295]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[296]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[297]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[298]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[299]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[300]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[301]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[302]	validation_0-rmse:3.40337	validation_1-rmse:3.40337
[303]	validati

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.5, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:
test['prediction'] = reg.predict(X_test)
test['prediction'] = test['prediction'].astype(int)
def roundValues(test):
    for i in range(len(test)):
        if(test.loc[i,'prediction']<0):
            test.loc[i,'prediction'] = 0
        else:
            test.loc[i,'prediction'] = round(math.ceil(test.loc[i,'prediction']),0)

In [15]:
roundValues(test)

#### Calculate Error

In [16]:
np.sqrt(mean_squared_error(test['Sum of NSQ'],test['prediction']))

3.4565577045174054

In [17]:
dataFrame = dataFrame.merge(test[['prediction']],how = 'left',left_index = True,right_index = True)
dataFrame

,Channel,Region,Display,Colour,Material,Sum of NSQ,prediction
0,2,2,1,0,5,0,0
1,2,2,1,0,7,0,0
2,2,4,1,0,5,7,6
3,2,9,5,0,5,1,1
4,2,9,5,0,1,2,1
...,...,...,...,...,...,...,...
742,0,3,3,4,3,1,1
743,0,3,3,4,9,0,0
744,0,3,3,5,3,0,0
745,0,3,3,7,3,0,0


#### Decode 

In [18]:
dataFrame['Channel'] =temp['Channel']  
dataFrame['Region']  =temp['Region']   
dataFrame['Display'] =temp['Display']  
dataFrame['Colour']  =temp['Colour']   
dataFrame['Material']=temp['Material'] 
dataFrame

,Channel,Region,Display,Colour,Material,Sum of NSQ,prediction
0,SIS,Chattisgarh,NF,BLACK,51/49 rPES/PU,0,0
1,SIS,Chattisgarh,NF,BLACK,60/40 REPREVE/40P POLYE,0,0
2,SIS,Gujarat,NF,BLACK,51/49 rPES/PU,7,6
3,SIS,Punjab,NaN,BLACK,51/49 rPES/PU,1,1
4,SIS,Punjab,NaN,BLACK,100% POLYESTER,2,1
...,...,...,...,...,...,...,...
742,FSS,Delhi,Wall,NAVY,100% rPES,1,1
743,FSS,Delhi,Wall,NAVY,80/20 NYLON/PU,0,0
744,FSS,Delhi,Wall,PURPLE,100% rPES,0,0
745,FSS,Delhi,Wall,RAZZLEBERRY,100% rPES,0,0


#### Save dataFrame

In [19]:
dataFrame.to_csv('XGBoost predictionBags.csv', index=False)